<a href="https://colab.research.google.com/github/timbosssds/ML/blob/main/Hyde_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This are extracts from Hyde (Kaggle)

In [2]:
# -- RAG set up

In [3]:
# --- Install lib
#!pip3 install --upgrade google-cloud-aiplatform
#!pip3 install ipython pandas[output_formatting] google-cloud-language==2.10.0
!pip install langchain
!pip install langchain-community

!pip install PyPDF2
!pip install pypdf
#!pip install sentence_transformers
!pip install sentence-transformers
!pip install torch
!pip install pandas
!pip install pdfplumber
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB

In [4]:
# -- Version 0a -- (trialing llama_index SentenceSplitter)
# Can i split by page + chunk

import os as os
import pandas as pd
pd.set_option('display.max_colwidth', 750)
from llama_index.core.node_parser import SentenceSplitter
import pdfplumber
from langchain.document_loaders import PDFPlumberLoader

def load_pdfs_from_directory(directory_path, max_chunk_size=1000):
    docs = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            loader = PDFPlumberLoader(file_path)
            loaded_docs = loader.load()

            # Use LlamaIndex SentenceSplitter
            text_splitter = SentenceSplitter()
            for doc in loaded_docs:
                sentences = text_splitter.split_text(doc.page_content)
                for sentence in sentences:
                    # Sub-chunk by character length (max 1000)
                    chunks = [sentence[i:i+max_chunk_size] for i in range(0, len(sentence), max_chunk_size)]
                    for chunk in chunks:
                        docs.append({
                            'chunk': chunk,  # Changed key to 'chunk'
                            'metadata': doc.metadata,
                            'len': len(chunk)
                    })
    return docs

#directory_path = "/home/jupyter/pdf"
#directory_path = "/kaggle/input/benpayfactsandcs/"
#directory_path = "/kaggle/input/hoemloanbooklet/"
directory_path = "/home"



all_pdf_docs = load_pdfs_from_directory(directory_path)

# Create DataFrame directly with desired columns
df = pd.DataFrame(all_pdf_docs)

# No further processing needed, df already has required columns

display(df.head(2))

,chunk,metadata,len
0,Bendigo Payment Facilities\nTerms and Conditions\n9 November 2023,"{'source': '/home/bendigo-payment-facilities-terms-conditions.pdf', 'file_path': '/home/bendigo-payment-facilities-terms-conditions.pdf', 'page': 0, 'total_pages': 50, 'Creator': 'Power PDF Create', 'Producer': 'Power PDF Create', 'CreationDate': 'D:20231107131823+10'00'', 'ModDate': 'D:20231108123535+10'00'', 'Author': 'Bianca Pisoni', 'Title': 'Bendigo-payment-facilities-terms-and-conditions-4-october-2022;', 'ContentTypeId': '0x010100E809092CA74C4245B60BDAEA41DE29DE', '_dlc_DocIdItemGuid': 'da16ccf0-5d9b-4890-8741-29ad99354354', 'PADocID': '39175925v1', 'MediaServiceImageTags': '', 'Subject': 'personal-payment-facilities-terms-and-conditions-4-october-2022;', 'Keywords': 'personal-payment-facilities-terms-and-conditions-4-october-2022'}",63
1,"Contents\nBendigo Payment Facilities Terms and Conditions 3\n1. These Terms and Conditions 3\n2. Codes of Practice 4\n3. Debit Cards 4\n4. Bendigo Phone Banking 7\n5. Bendigo e-banking 8\n6. Internal Transfers 11\n7. Pay Anyone (including Osko and Fast Payments) 11\n8. PayIDs 16\n9. PayTo 19\n10. BPAY and BPAY View 25\n11. Automatic Payments 31\n12. Telegraphic Transfers 32\n13. Transaction limits and processing 34\n14. Records of transactions 35\n15. Disputing transactions and refunds and chargebacks 35\n16. Keeping your payment devices secure 36\n17. Liability for Transactions 38\n18. Our liability to you 39\n19. Suspending, blocking and cancelling your Payment Facility 40\n20. How we will communicate with you 42\n21. How you can comm...","{'source': '/home/bendigo-payment-facilities-terms-conditions.pdf', 'file_path': '/home/bendigo-payment-facilities-terms-conditions.pdf', 'page': 1, 'total_pages': 50, 'Creator': 'Power PDF Create', 'Producer': 'Power PDF Create', 'CreationDate': 'D:20231107131823+10'00'', 'ModDate': 'D:20231108123535+10'00'', 'Author': 'Bianca Pisoni', 'Title': 'Bendigo-payment-facilities-terms-and-conditions-4-october-2022;', 'ContentTypeId': '0x010100E809092CA74C4245B60BDAEA41DE29DE', '_dlc_DocIdItemGuid': 'da16ccf0-5d9b-4890-8741-29ad99354354', 'PADocID': '39175925v1', 'MediaServiceImageTags': '', 'Subject': 'personal-payment-facilities-terms-and-conditions-4-october-2022;', 'Keywords': 'personal-payment-facilities-terms-and-conditions-4-october-2022'}",885


In [5]:
# # -- Investigation (for Version 0a)
df1 = df.copy()
print('shape', df1.shape)
pd.set_option('display.max_colwidth', None)
dfa = df1[['chunk','metadata', 'len']]
dfa['chunk_org'] = dfa['chunk'].str.lower()
df_filtered = dfa[dfa['chunk_org'].str.contains('easy money')]
display(df_filtered.head())

shape (273, 3)


,chunk,metadata,len,chunk_org
5,"responsibility on you unless permitted by the ePayments Code.\n3. Debit Cards\nThis ‘Debit Cards’ section applies in relation to any Debit Cards we issue to you or at your request or\nallow you to use in connection with your Account.\nThe types of Debit Cards we offer are:\nDebit Card type Bendigo Debit Bendigo Easy Money Bendigo Youth Debit\nMastercard Card Mastercard\nEligibility 16 years of age or older 12 years of age or older 12-15 years of age\nAccess Anywhere Mastercard is Anywhere EFTPOS is Anywhere Mastercard is\naccepted, including ATM accepted, including ATM accepted, including ATM\nwithdrawals. withdrawals. withdrawals.\nOverseas access Overseas ATM access No overseas access Overseas ATM access\navailable at any ATM available. available at any ATM\nbearing the Mastercard, bearing the Mastercard,\nCIRRUS, or MAESTRO CIRRUS, or MAESTRO\nsymbol. symbol.\nGeneral / other Restrictions apply to\ncriteria, features or\nTransactions at certain\nrestrictions\ntypes of merchants (see\n‘Youth Debit Maste","{'source': '/home/bendigo-payment-facilities-terms-conditions.pdf', 'file_path': '/home/bendigo-payment-facilities-terms-conditions.pdf', 'page': 3, 'total_pages': 50, 'Creator': 'Power PDF Create', 'Producer': 'Power PDF Create', 'CreationDate': 'D:20231107131823+10'00'', 'ModDate': 'D:20231108123535+10'00'', 'Author': 'Bianca Pisoni', 'Title': 'Bendigo-payment-facilities-terms-and-conditions-4-october-2022;', 'ContentTypeId': '0x010100E809092CA74C4245B60BDAEA41DE29DE', '_dlc_DocIdItemGuid': 'da16ccf0-5d9b-4890-8741-29ad99354354', 'PADocID': '39175925v1', 'MediaServiceImageTags': '', 'Subject': 'personal-payment-facilities-terms-and-conditions-4-october-2022;', 'Keywords': 'personal-payment-facilities-terms-and-conditions-4-october-2022'}",1000,"responsibility on you unless permitted by the epayments code.\n3. debit cards\nthis ‘debit cards’ section applies in relation to any debit cards we issue to you or at your request or\nallow you to use in connection with your account.\nthe types of debit cards we offer are:\ndebit card type bendigo debit bendigo easy money bendigo youth debit\nmastercard card mastercard\neligibility 16 years of age or older 12 years of age or older 12-15 years of age\naccess anywhere mastercard is anywhere eftpos is anywhere mastercard is\naccepted, including atm accepted, including atm accepted, including atm\nwithdrawals. withdrawals. withdrawals.\noverseas access overseas atm access no overseas access overseas atm access\navailable at any atm available. available at any atm\nbearing the mastercard, bearing the mastercard,\ncirrus, or maestro cirrus, or maestro\nsymbol. symbol.\ngeneral / other restrictions apply to\ncriteria, features or\ntransactions at certain\nrestrictions\ntypes of merchants (see\n‘youth debit maste"
158,"Bendigo e-banking Our internet banking service we make accessible through a web\nbrowser at www.bendigobank.com.au or by using a Bendigo Bank\napp.\nBendigo Phone Banking Our Bendigo Phone Banking service we make available to you by\ncalling 1300 236 344.\nBPAY BPAY Pty Ltd ABN 69 079 137 518.\nBPAY Group Limited BPAY Group Pty Ltd ABN 60 003 311 644.\nBPAY Payment A payment which you have instructed us to make through BPAY\nPayments to billers who can accept payments made to them through\nthat scheme.\nBPAY Scheme The Scheme operated by BPAY which governs BPAY Payments and\nOsko.\nBSB The Bank/State/Branch number associated with your Account.\nCard A debit card or credit card we have issued.\nCard Scheme The payment network relevant to a Card.\nDebit Card A Debit Mastercard, Youth Debit Mastercard, or an Easy Money card.\nIn these Terms and Conditions, ‘Debit Card’ also includes a credit card\nthat we issue where we have setup the card to allow you to use it to\naccess an Account (for example, if we li","{'source': '/home/bendigo-payment-facilities-terms-conditions.pdf', 'file_path': '/home/bendigo-payment-facilities-terms-conditions.pdf', '